In [25]:
#!/usr/bin/env python3
"""
Correlation test between ViT and Mouse Brain PCA components.

 - Tests H0: corr(ViT PC_i, Brain PC_j) = 0
 - Uses all PCs explaining ≥90% variance in each space
 - Applies Benjamini–Hochberg correction for multiple testing
 - Prints significant pairs and summary statistics
"""

import os
import pickle
import numpy as np
from sklearn.decomposition import PCA
from scipy.special import softmax
from skbio.stats.composition import clr
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

# ---------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------
VIT_PATH = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'       # brain area label per neuron
AREA_NAME = 'VISpm'                 # choose brain area
N_IMAGES, N_TRIALS = 118, 50
ALPHA_FDR = 0.05

# ---------------------------------------------------------------
# LOAD ViT EMBEDDINGS AND RUN PCA
# ---------------------------------------------------------------
print("🔹 Loading ViT embeddings...")
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']
embeddings = np.asarray(vit_logits)
X = softmax(embeddings, axis=1)
X_clr = clr(X + 1e-12)

pca_vit_full = PCA(n_components=min(X_clr.shape), random_state=0)
pca_vit_full.fit(X_clr)
vit_cumvar = np.cumsum(pca_vit_full.explained_variance_ratio_)
vit_ncomp = np.searchsorted(vit_cumvar, 0.90) + 1
print(f"ViT PCs covering 90% variance: {vit_ncomp}")

vit_pca = PCA(n_components=vit_ncomp, random_state=0)
vit_U = vit_pca.fit_transform(X_clr)      # (images × vit_ncomp)
vit_var = vit_pca.explained_variance_ratio_

# ---------------------------------------------------------------
# LOAD AND FILTER BRAIN DATA
# ---------------------------------------------------------------
print(f"🔹 Loading neural responses ({AREA_NAME})...")
dat = np.load(NEURAL_PATH, mmap_mode='r')
areas = np.load(AREAS_PATH, allow_pickle=True)
mask = areas == AREA_NAME
dat = dat[mask]
n_neurons, n_total = dat.shape
n_time = n_total // (N_IMAGES * N_TRIALS)
dat = dat.reshape(n_neurons, N_IMAGES, N_TRIALS, n_time)
X_mean = dat.mean(axis=(2, 3))  # (neurons × images)

# ---------------------------------------------------------------
# PCA ON BRAIN DATA
# ---------------------------------------------------------------
pca_brain_full = PCA(n_components=min(X_mean.shape), random_state=0)
pca_brain_full.fit(X_mean - X_mean.mean(axis=0))
brain_cumvar = np.cumsum(pca_brain_full.explained_variance_ratio_)
brain_ncomp = np.searchsorted(brain_cumvar, 0.90) + 1
print(f"{AREA_NAME} PCs covering 90% variance: {brain_ncomp}")

pca_brain = PCA(n_components=brain_ncomp, random_state=0)
brain_V = pca_brain.fit_transform(X_mean - X_mean.mean(axis=0))  # (neurons × brain_ncomp)
brain_loadings = pca_brain.components_.T                         # (images × brain_ncomp)

# ---------------------------------------------------------------
# CORRELATION TESTS
# ---------------------------------------------------------------
print("🔹 Computing all pairwise correlations...")
r_vals, p_vals, pairs = [], [], []

for i in range(vit_ncomp):
    for j in range(brain_ncomp):
        r, p = pearsonr(vit_U[:, i], brain_loadings[:, j])
        r_vals.append(r)
        p_vals.append(p)
        pairs.append((i+1, j+1))  # 1-indexed for reporting

r_vals = np.array(r_vals)
p_vals = np.array(p_vals)

n_tests = len(p_vals)
print(f"Total tests performed: {n_tests}")

# ---------------------------------------------------------------
# MULTIPLE TESTING CORRECTION
# ---------------------------------------------------------------
reject, pvals_corrected, _, _ = multipletests(p_vals, alpha=ALPHA_FDR, method='fdr_bh')

sig_idx = np.where(reject)[0]
if len(sig_idx) == 0:
    print("❌ No correlations survived FDR correction.")
else:
    print(f"✅ {len(sig_idx)} / {n_tests} correlations survived FDR correction (q < {ALPHA_FDR}).")
    print("\nSignificant pairs (ViT PC ↔ Brain PC):")
    for idx in sig_idx:
        i, j = pairs[idx]
        print(f"  ViT PC{i:2d} ↔ {AREA_NAME} PC{j:2d} | r = {r_vals[idx]:+.3f}, p = {p_vals[idx]:.2e}, q = {pvals_corrected[idx]:.2e}")

# ---------------------------------------------------------------
# SAVE RESULTS
# ---------------------------------------------------------------
out = {
    "pairs": pairs,
    "r": r_vals,
    "p_raw": p_vals,
    "p_fdr": pvals_corrected,
    "significant": reject,
    "vit_ncomp": vit_ncomp,
    "brain_ncomp": brain_ncomp,
}
np.savez(f"vit_{AREA_NAME}_corr_results.npz", **out)
print(f"\n💾 Results saved to vitpm PC pairs (q < 0.05)_{AREA_NAME}_corr_results.npz")


🔹 Loading ViT embeddings...
ViT PCs covering 90% variance: 44
🔹 Loading neural responses (VISpm)...
VISpm PCs covering 90% variance: 69
🔹 Computing all pairwise correlations...
Total tests performed: 3036
✅ 2 / 3036 correlations survived FDR correction (q < 0.05).

Significant pairs (ViT PC ↔ Brain PC):
  ViT PC 1 ↔ VISpm PC 4 | r = +0.436, p = 7.87e-07, q = 2.39e-03
  ViT PC34 ↔ VISpm PC58 | r = -0.410, p = 4.01e-06, q = 6.09e-03

💾 Results saved to vitpm PC pairs (q < 0.05)_VISpm_corr_results.npz


In [ ]:
# ---------------------------------------------------------------
# 5-FOLD CROSS-VALIDATION ON SIGNIFICANT PC PAIRS (FIXED FINAL)
# ---------------------------------------------------------------
from sklearn.model_selection import KFold

if np.any(reject):
    print("\n🔹 Running 5-fold cross-validation on significant PC pairs...")

    kf = KFold(n_splits=2, shuffle=True, random_state=42)
    cv_results = {tuple(pairs[idx]): [] for idx in sig_idx}

    for fold, (train_idx, test_idx) in enumerate(kf.split(np.arange(N_IMAGES)), 1):
        # ---- Split data by image ----
        vit_train, vit_test = X_clr[train_idx], X_clr[test_idx]
        brain_train, brain_test = X_mean[:, train_idx], X_mean[:, test_idx]

        # ---- ViT PCA ----
        vit_pca = PCA(n_components=vit_ncomp, random_state=0)
        vit_U_train = vit_pca.fit_transform(vit_train)
        vit_U_test = vit_pca.transform(vit_test)

        # ---- Brain PCA (note the .T for samples = images) ----
        brain_pca = PCA(n_components=brain_ncomp, random_state=0)

        # center each neuron's responses (rows) using training means
        train_mean_per_neuron = brain_train.mean(axis=1, keepdims=True)
        brain_train_centered = (brain_train - train_mean_per_neuron).T  # (images_train × neurons)
        brain_test_centered  = (brain_test  - train_mean_per_neuron).T  # (images_test × neurons)

        # Fit PCA on training images × neurons matrix
        brain_pca.fit(brain_train_centered)

        # Transform test data to get brain PC scores (images_test × ncomp)
        brain_scores_test = brain_pca.transform(brain_test_centered)

        # ---- Compute correlations for significant pairs ----
        for (i, j) in cv_results.keys():
            try:
                r, _ = pearsonr(vit_U_test[:, i-1], brain_scores_test[:, j-1])
            except Exception:
                r = np.nan
            cv_results[(i, j)].append(r)

        print(f"  Fold {fold} done.")

    # ---- Summarize ----
    summary = []
    for (i, j), rlist in cv_results.items():
        rlist = np.array(rlist)
        mean_r, std_r = np.nanmean(rlist), np.nanstd(rlist)
        summary.append((i, j, mean_r, std_r))

    summary = np.array(summary, dtype=[("vit_pc", int),
                                       ("brain_pc", int),
                                       ("r_mean_cv", float),
                                       ("r_std_cv", float)])

    np.save(f"vit_{AREA_NAME}_cv_summary.npy", summary)

    print("\n✅ Cross-validation complete. Results (mean ± std):")
    for row in sorted(summary, key=lambda x: -abs(x["r_mean_cv"]))[:10]:
        print(f"  ViT PC{row['vit_pc']:2d} ↔ {AREA_NAME} PC{row['brain_pc']:2d} | "
              f"r_cv = {row['r_mean_cv']:+.3f} ± {row['r_std_cv']:.3f}")

    print(f"\n💾 Saved to vit_{AREA_NAME}_cv_summary.npy")

else:
    print("\n⚠️ No significant pairs to cross-validate.")



🔹 Running 5-fold cross-validation on significant PC pairs...


ValueError: n_components=69 must be between 0 and min(n_samples, n_features)=59 with svd_solver='full'

In [26]:
#!/usr/bin/env python3
"""
5-fold cross-validation for significant ViT–Brain PC correlations.

Given the significant pairs from the main correlation script, this tests
whether those relationships generalize to unseen images.

Author: Maria + Pl2ku 🐾
"""

import numpy as np
import pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scipy.special import softmax
from scipy.stats import pearsonr
from skbio.stats.composition import clr
from statsmodels.stats.multitest import multipletests

# ---------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------
VIT_PATH = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'
AREA_NAME = 'VISam'                 # choose brain area
N_IMAGES, N_TRIALS = 118, 50
ALPHA_FDR = 0.05

# ---------------------------------------------------------------
# LOAD ViT EMBEDDINGS + PCA
# ---------------------------------------------------------------
print("🔹 Loading ViT embeddings...")
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']

embeddings = np.asarray(vit_logits)
X = softmax(embeddings, axis=1)
X_clr = clr(X + 1e-12)

pca_vit_full = PCA(n_components=min(X_clr.shape), random_state=0)
pca_vit_full.fit(X_clr)
vit_cumvar = np.cumsum(pca_vit_full.explained_variance_ratio_)
vit_ncomp = np.searchsorted(vit_cumvar, 0.90) + 1
print(f"ViT PCs covering 90% variance: {vit_ncomp}")

pca_vit = PCA(n_components=vit_ncomp, random_state=0)
vit_U = pca_vit.fit_transform(X_clr)  # (images × vit_ncomp)

# ---------------------------------------------------------------
# LOAD AND FILTER BRAIN DATA
# ---------------------------------------------------------------
print(f"🔹 Loading neural responses ({AREA_NAME})...")
dat = np.load(NEURAL_PATH, mmap_mode='r')
areas = np.load(AREAS_PATH, allow_pickle=True)
mask = areas == AREA_NAME
dat = dat[mask]
n_neurons, n_total = dat.shape
n_time = n_total // (N_IMAGES * N_TRIALS)
dat = dat.reshape(n_neurons, N_IMAGES, N_TRIALS, n_time)
X_mean = dat.mean(axis=(2, 3))  # (neurons × images)
print(f"{n_neurons} neurons × {N_IMAGES} images")

# ---------------------------------------------------------------
# BRAIN PCA
# ---------------------------------------------------------------
pca_brain_full = PCA(n_components=min(X_mean.shape), random_state=0)
pca_brain_full.fit(X_mean - X_mean.mean(axis=0))
brain_cumvar = np.cumsum(pca_brain_full.explained_variance_ratio_)
brain_ncomp = np.searchsorted(brain_cumvar, 0.90) + 1
print(f"{AREA_NAME} PCs covering 90% variance: {brain_ncomp}")

pca_brain = PCA(n_components=brain_ncomp, random_state=0)
brain_loadings = pca_brain.fit_transform((X_mean - X_mean.mean(axis=0)).T)  # (images × brain_ncomp)

# ---------------------------------------------------------------
# VIΤ ↔ BRAIN CORRELATION TESTS
# ---------------------------------------------------------------
print("🔹 Computing all pairwise correlations...")
r_vals, p_vals, pairs = [], [], []
for i in range(vit_ncomp):
    for j in range(brain_ncomp):
        r, p = pearsonr(vit_U[:, i], brain_loadings[:, j])
        r_vals.append(r)
        p_vals.append(p)
        pairs.append((i + 1, j + 1))

r_vals, p_vals = np.array(r_vals), np.array(p_vals)
reject, pvals_corrected, _, _ = multipletests(p_vals, alpha=ALPHA_FDR, method='fdr_bh')

sig_idx = np.where(reject)[0]
if not len(sig_idx):
    print("❌ No correlations survived FDR correction.")
    exit(0)

print(f"✅ {len(sig_idx)} significant ViT–{AREA_NAME} PC pairs (q < {ALPHA_FDR})")
sig_pairs = [pairs[i] for i in sig_idx]

# ---------------------------------------------------------------
# 5-FOLD CROSS-VALIDATION
# ---------------------------------------------------------------
print("\n🔹 Running 5-fold cross-validation on significant pairs...")
kf = KFold(n_splits=3, shuffle=True, random_state=42)
cv_results = {tuple(p): [] for p in sig_pairs}

for fold, (train_idx, test_idx) in enumerate(kf.split(np.arange(N_IMAGES)), 1):
    # ---- Split images ----
    vit_train, vit_test = X_clr[train_idx], X_clr[test_idx]
    brain_train, brain_test = X_mean[:, train_idx], X_mean[:, test_idx]

    # ---- ViT PCA ----
    vit_pca = PCA(n_components=vit_ncomp, random_state=0)
    vit_U_train = vit_pca.fit_transform(vit_train)
    vit_U_test = vit_pca.transform(vit_test)

    # ---- Brain PCA (train images × neurons) ----
    train_mean_per_neuron = brain_train.mean(axis=1, keepdims=True)
    brain_train_centered = (brain_train - train_mean_per_neuron).T
    brain_test_centered  = (brain_test  - train_mean_per_neuron).T

    n_comps_fold = min(brain_ncomp,
                       brain_train_centered.shape[0],
                       brain_train_centered.shape[1])
    brain_pca = PCA(n_components=n_comps_fold, random_state=0)
    brain_pca.fit(brain_train_centered)
    brain_scores_test = brain_pca.transform(brain_test_centered)

    print(f"  Fold {fold}: using {n_comps_fold} brain PCs ({len(train_idx)} train imgs)")

    # ---- Correlations for significant pairs ----
    for (i, j) in cv_results.keys():
        if j > n_comps_fold:
            cv_results[(i, j)].append(np.nan)
            continue
        try:
            r, _ = pearsonr(vit_U_test[:, i - 1], brain_scores_test[:, j - 1])
        except Exception:
            r = np.nan
        cv_results[(i, j)].append(r)

# ---------------------------------------------------------------
# SUMMARY
# ---------------------------------------------------------------
summary = []
for (i, j), rlist in cv_results.items():
    rlist = np.array(rlist, float)
    mean_r, std_r = np.nanmean(rlist), np.nanstd(rlist)
    summary.append((i, j, mean_r, std_r))

summary = np.array(summary,
                   dtype=[("vit_pc", int), ("brain_pc", int),
                          ("r_mean_cv", float), ("r_std_cv", float)])
np.save(f"vit_{AREA_NAME}_cv_summary.npy", summary)

print("\n✅ Cross-validation complete. Results (mean ± std):")
for row in sorted(summary, key=lambda x: -abs(x["r_mean_cv"]))[:10]:
    print(f"  ViT PC{row['vit_pc']:2d} ↔ {AREA_NAME} PC{row['brain_pc']:2d} "
          f"| r_cv = {row['r_mean_cv']:+.3f} ± {row['r_std_cv']:.3f}")

print(f"\n💾 Saved to vit_{AREA_NAME}_cv_summary.npy")


🔹 Loading ViT embeddings...
ViT PCs covering 90% variance: 44
🔹 Loading neural responses (VISam)...
2040 neurons × 118 images
VISam PCs covering 90% variance: 61
🔹 Computing all pairwise correlations...
✅ 1 significant ViT–VISam PC pairs (q < 0.05)

🔹 Running 5-fold cross-validation on significant pairs...
  Fold 1: using 61 brain PCs (78 train imgs)
  Fold 2: using 61 brain PCs (79 train imgs)
  Fold 3: using 61 brain PCs (79 train imgs)

✅ Cross-validation complete. Results (mean ± std):
  ViT PC 1 ↔ VISam PC 1 | r_cv = +0.356 ± 0.042

💾 Saved to vit_VISam_cv_summary.npy


In [27]:
sim = np.abs(np.dot(brain_pca_full.components_, brain_pca_fold.components_.T))

NameError: name 'brain_pca_full' is not defined

In [28]:
#!/usr/bin/env python3
"""
Fixed-basis cross-validation for ViT–Brain PC correlations.

 - PCA bases are fit once on the full dataset (no re-fitting per fold)
 - Cross-validation only tests generalization of *the same axes*
 - Avoids leakage in correlation test, but keeps PC indexing stable
"""

import numpy as np
import pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scipy.special import softmax
from scipy.stats import pearsonr
from skbio.stats.composition import clr
from statsmodels.stats.multitest import multipletests

# ---------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------
VIT_PATH   = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH  = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'
AREA_NAME   = 'VISpm'
N_IMAGES, N_TRIALS = 118, 50
ALPHA_FDR = 0.05
N_SPLITS  = 2     # 2-fold or whatever you like

# ---------------------------------------------------------------
# LOAD ViT EMBEDDINGS + PCA (full data)
# ---------------------------------------------------------------
print("🔹 Loading ViT embeddings...")
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']

vit_soft = softmax(np.asarray(vit_logits), axis=1)
vit_clr  = clr(vit_soft + 1e-12)

pca_vit_full = PCA(random_state=0)
pca_vit_full.fit(vit_clr)
vit_cum = np.cumsum(pca_vit_full.explained_variance_ratio_)
vit_ncomp = np.searchsorted(vit_cum, 0.90) + 1
print(f"ViT PCs covering 90% variance: {vit_ncomp}")
W_vit = pca_vit_full.components_[:vit_ncomp]        # frozen ViT basis
vit_U_full = vit_clr @ W_vit.T                      # full-data scores

# ---------------------------------------------------------------
# LOAD BRAIN DATA + PCA (full data)
# ---------------------------------------------------------------
print(f"🔹 Loading neural responses ({AREA_NAME})...")
dat = np.load(NEURAL_PATH, mmap_mode='r')
areas = np.load(AREAS_PATH, allow_pickle=True)
mask = areas == AREA_NAME
dat  = dat[mask]
n_neurons, n_total = dat.shape
n_time = n_total // (N_IMAGES * N_TRIALS)
dat = dat.reshape(n_neurons, N_IMAGES, N_TRIALS, n_time)
X_mean = dat.mean(axis=(2,3))                       # neurons × images

pca_brain_full = PCA(random_state=0)
pca_brain_full.fit(X_mean.T)
brain_cum = np.cumsum(pca_brain_full.explained_variance_ratio_)
brain_ncomp = np.searchsorted(brain_cum, 0.90) + 1
print(f"{AREA_NAME} PCs covering 90% variance: {brain_ncomp}")
W_brain = pca_brain_full.components_[:brain_ncomp]  # frozen brain basis
brain_scores_full = X_mean.T @ W_brain.T            # images × brain_ncomp

# ---------------------------------------------------------------
# INITIAL DISCOVERY (on full data)
# ---------------------------------------------------------------
print("🔹 Full-data discovery correlations...")
r_vals, p_vals, pairs = [], [], []
for i in range(vit_ncomp):
    for j in range(brain_ncomp):
        r, p = pearsonr(vit_U_full[:, i], brain_scores_full[:, j])
        r_vals.append(r); p_vals.append(p)
        pairs.append((i+1, j+1))
r_vals, p_vals = np.array(r_vals), np.array(p_vals)
reject, pvals_corr, _, _ = multipletests(p_vals, alpha=ALPHA_FDR, method='fdr_bh')
sig_pairs = [pairs[k] for k in np.where(reject)[0]]
print(f"✅ {len(sig_pairs)} significant pairs (q<{ALPHA_FDR})")

# ---------------------------------------------------------------
# CROSS-VALIDATION (fixed bases)
# ---------------------------------------------------------------
print("\n🔹 Running cross-validation with fixed PCA bases...")
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
cv_results = {tuple(p): [] for p in sig_pairs}

for fold, (train_idx, test_idx) in enumerate(kf.split(np.arange(N_IMAGES)), 1):
    vit_test   = vit_clr[test_idx]
    brain_test = X_mean[:, test_idx].T  # images × neurons

    # project test data into the fixed bases
    vit_scores_test   = vit_test @ W_vit.T
    brain_scores_test = brain_test @ W_brain.T

    print(f"  Fold {fold}: {len(test_idx)} test images")

    for (i,j) in sig_pairs:
        r,_ = pearsonr(vit_scores_test[:, i-1], brain_scores_test[:, j-1])
        cv_results[(i,j)].append(r)

# ---------------------------------------------------------------
# SUMMARY
# ---------------------------------------------------------------
summary=[]
for (i,j),rlist in cv_results.items():
    arr=np.array(rlist,float)
    summary.append((i,j,np.nanmean(arr),np.nanstd(arr)))
summary=np.array(summary,dtype=[("vit_pc",int),("brain_pc",int),
                                ("r_mean_cv",float),("r_std_cv",float)])
np.save(f"vit_{AREA_NAME}_cv_fixedbasis_summary.npy",summary)

print("\n✅ Cross-validation (fixed basis) complete:")
for row in sorted(summary,key=lambda x:-abs(x["r_mean_cv"]))[:10]:
    print(f"  ViT PC{row['vit_pc']:2d} ↔ {AREA_NAME} PC{row['brain_pc']:2d}"
          f" | r_cv = {row['r_mean_cv']:+.3f} ± {row['r_std_cv']:.3f}")

print(f"\n💾 Saved to vit_{AREA_NAME}_cv_fixedbasis_summary.npy")


🔹 Loading ViT embeddings...
ViT PCs covering 90% variance: 44
🔹 Loading neural responses (VISpm)...
VISpm PCs covering 90% variance: 83
🔹 Full-data discovery correlations...
✅ 2 significant pairs (q<0.05)

🔹 Running cross-validation with fixed PCA bases...
  Fold 1: 59 test images
  Fold 2: 59 test images

✅ Cross-validation (fixed basis) complete:
  ViT PC 1 ↔ VISpm PC 3 | r_cv = +0.432 ± 0.066
  ViT PC34 ↔ VISpm PC57 | r_cv = +0.401 ± 0.039

💾 Saved to vit_VISpm_cv_fixedbasis_summary.npy
